In [1]:
import os

In [2]:
%pwd

'e:\\ML Folder\\chicken_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\ML Folder\\chicken_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [7]:
from src.cnnClassifier.constants import *


In [8]:
from src.cnnClassifier.utils.common import read_yaml,create_directories

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH
    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingetion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier.logging import logger
from src.cnnClassifier.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded")
        else:
            logger.info(f"{self.config.local_data_file} already exists")

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingetion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-03 14:21:22,726:INFO:common:yaml_file:config\config.yaml loaded successfully]
[2025-03-03 14:21:22,731:INFO:common:yaml_file:params.yaml loaded successfully]
[2025-03-03 14:21:22,735:INFO:common:created directoty at : artifacts]
[2025-03-03 14:21:22,739:INFO:common:created directoty at : artifacts/data_ingestion]
[2025-03-03 14:21:34,267:INFO:3529762362:artifacts/data_ingestion/data.zip downloaded]
